## contains code that will create the necessary .csv files

In [1]:
from json import load
import pandas as pd
import numpy as np
import os
import time

In [2]:
folder = r"./ipl_json" 
# include the path to the folder ipl_json here

In [3]:
def get_number_of_deliveries(inning):
    x = 0
    for over in inning:
        for j in range(len(over["deliveries"])):
            x +=1
    return x

In [4]:
def get_pos(pos, inning):
    x = []
    for over in inning:
        for j in range(len(over["deliveries"])):
            
            x.append(over["deliveries"][j].get(pos))
    return x

In [5]:
def check_if_player_in_match(player,match):
    return any(player in team for team in match["info"]["players"].values())

In [6]:
def get_team_of_player(player, match):
    team = [team for team, players in match["info"]["players"].items() if player in players]
    return team

In [7]:
def get_runs(way, deliveries):
    x = []
    for i in deliveries:
        x.append(i["runs"]["total"])
    return x

In [8]:
def get_wicket_info(inning):
    x = []
    fielders = []
    batter_out = []
    wicket_kind = []
    is_wicket_delivery = []
    overs = []
    
    over_count = 1
    for over in inning:
        for j in range(len(over["deliveries"])):
            
            if "wickets" in over["deliveries"][j].keys():
                ball = over["deliveries"][j]["wickets"][0]
                
                batter_out.append(ball["player_out"])
                wicket_kind.append(ball["kind"])
                is_wicket_delivery.append(1)
                overs.append(over_count)
                
                if "fielders" in ball.keys():
                    z = []
                    if len(ball["fielders"]) > 1:
                        for k in ball["fielders"]:
                            z.append(list(k.values())[0])
                    else:
                        z = list(ball["fielders"][0].values())
                    fielders.append(z)
                else:
                    fielders.append(0)
            else:
                    fielders.append(0)
                    batter_out.append(0)
                    wicket_kind.append(0)
                    is_wicket_delivery.append(0)
                    overs.append(over_count)
        over_count += 1

    return overs,is_wicket_delivery, wicket_kind, batter_out, fielders

In [9]:
def reader(file): 
    # just to test how the data works
    with open(folder+"\\"+ file, "r") as f_input:
        
        data = load(f_input)
        innings = data['innings']
        innings_name = ["1st innings", "2nd innings", "3rd innings", "4th innings"]
        list0,list1 = [],[]
        
        for i in range(len(innings)):
            dict_innings = {}
            try:
                inning = innings[i]["overs"]
            except:
                print("boo")
                continue
            
    return data

In [12]:
data = reader("335982.json")

In [13]:
data["info"]["dates"][0]

'2008-04-18'

In [14]:
if "result" in data["info"]["outcome"]:
    if data["info"]["outcome"]["result"] == "tie":
        print("winner: "  , data["info"]["outcome"]["eliminator"])
    else:
        print("no result")
else:
    print(data["info"]["outcome"])
    

{'by': {'runs': 140}, 'winner': 'Kolkata Knight Riders'}


In [15]:
def framemaker():
#     makes the dataframe from the json files
    ballbyball = pd.DataFrame()
    for file in os.listdir(r".\ipl_json"):
        with open(folder+"\\" + file, "r") as f_input:
            match_id = file.rstrip(".json")
            
            match_frame = pd.DataFrame() 
            match_data = load(f_input)
            
            innings = match_data["innings"]
            iswickets,kind, batter, fielders, overs = [],[],[],[],[]
            participating_teams = match_data["info"]["teams"]
            
#             fields to be added in the dataframe , apart from the ball info 
#             like meta-data for the match/inning
            
            batting_team_series = []
            bowling_team_series = []
            inning_series = []
            match_series = []
            year_series = []
            winner_series = []
            margin_series = []
            
            year = match_data["info"]["dates"][0]
            year = year[:year.index("-")]
            outcome = match_data["info"]["outcome"]
            
            for i in range(len(innings)):

                inning = innings[i]
                batting_team = inning["team"]
                bowling_team = [i for i in participating_teams if i != batting_team][0]

                df = pd.DataFrame()
                for over in range(len(inning["overs"])):
                    for ball in inning["overs"][over]["deliveries"]:
                        match_series.append(match_id)
                        batting_team_series.append(batting_team)
                        bowling_team_series.append(bowling_team)
                        inning_series.append(i+1)
                        
                        if "result" in outcome:
                            if outcome["result"] == "tie":
                                margin_series.append("tie")
                                winner_series.append(outcome["eliminator"])
                            else:
                                margin_series.append("tie")                    
                                winner_series.append("no result")
                        else:
                                margin_series.append(outcome["by"])
                                winner_series.append(outcome["winner"])
                                
                        year_series.append(year)
                        
#                         flattens the ball-info , easiest way to do it
                        t = pd.json_normalize(ball) 
                        df = pd.concat([df,t])
                u,v,x,y,z = get_wicket_info(inning["overs"])
                
                overs.extend(u)
                iswickets.extend(v)
                kind.extend(x)
                batter.extend(y)
                fielders.extend(z)
                
                match_frame = pd.concat([match_frame, df])

                match_frame["isWicketDelivery"] = iswickets
                match_frame["wicketKind"] = kind
                match_frame["batterOut"] = batter
                match_frame["fieldersInvolved"] = fielders
                match_frame["over"] = overs
                
                match_frame["innings"] = inning_series
                match_frame["BattingTeam"] = batting_team_series
                match_frame["BowlingTeam"] = bowling_team_series
                match_frame["MatchID"] = match_series
                match_frame["year"] = year_series                
                match_frame["margin"] = margin_series
                match_frame["winner"] = winner_series
                
            ballbyball = pd.concat([ballbyball,match_frame])
    return ballbyball

In [16]:
start_time = time.time()
l = framemaker()
end_time = time.time()

In [17]:
end_time - start_time

347.65874552726746

In [18]:
useful_columns = ["MatchID","year" , 'over', 'innings','batter', 'bowler', 'non_striker', 'runs.batter', 'runs.extras',
       'runs.total', "extras.wides", "extras.legbyes", "extras.noballs" ,"extras.byes",
       'isWicketDelivery', 'wicketKind', 'batterOut', 'fieldersInvolved',
        'BattingTeam' , "BowlingTeam" , "winner", "margin"]

In [19]:
li = l[useful_columns]

In [20]:
li = li.fillna(0)

In [21]:
li.to_csv("ipl-ball-by-ball.csv")

In [49]:
from ast import literal_eval
import json
df = pd.read_csv("names.csv",converters={'team': literal_eval})

In [50]:
names = pd.read_csv("names.csv")
people = pd.read_csv("people.csv")

people = people[["identifier", "name" , "unique_name"]]
names = names.rename(columns = {"identifier":"identifier", "name": "cname"})



In [51]:
df = pd.merge(names, people, how = "left", left_on="identifier", right_on="identifier")

In [52]:
df.isnull().sum()

identifier     0
cname          0
name           0
unique_name    0
dtype: int64

In [53]:
db = pd.read_csv("ipl-ball-by-ball.csv")

In [54]:
db

,Unnamed: 0,MatchID,year,over,innings,batter,bowler,non_striker,runs.batter,runs.extras,...,extras.noballs,extras.byes,isWicketDelivery,wicketKind,batterOut,fieldersInvolved,BattingTeam,BowlingTeam,winner,margin
0,0,1082591,2017,1,1,DA Warner,TS Mills,S Dhawan,0,0,...,0.0,0.0,0,0,0,0,Sunrisers Hyderabad,Royal Challengers Bangalore,Sunrisers Hyderabad,{'runs': 35}
1,0,1082591,2017,1,1,DA Warner,TS Mills,S Dhawan,0,0,...,0.0,0.0,0,0,0,0,Sunrisers Hyderabad,Royal Challengers Bangalore,Sunrisers Hyderabad,{'runs': 35}
2,0,1082591,2017,1,1,DA Warner,TS Mills,S Dhawan,4,0,...,0.0,0.0,0,0,0,0,Sunrisers Hyderabad,Royal Challengers Bangalore,Sunrisers Hyderabad,{'runs': 35}
3,0,1082591,2017,1,1,DA Warner,TS Mills,S Dhawan,0,0,...,0.0,0.0,0,0,0,0,Sunrisers Hyderabad,Royal Challengers Bangalore,Sunrisers Hyderabad,{'runs': 35}
4,0,1082591,2017,1,1,DA Warner,TS Mills,S Dhawan,0,2,...,0.0,0.0,0,0,0,0,Sunrisers Hyderabad,Royal Challengers Bangalore,Sunrisers Hyderabad,{'runs': 35}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243812,0,981019,2016,20,2,Sachin Baby,B Kumar,CJ Jordan,2,0,...,0.0,0.0,0,0,0,0,Royal Challengers Bangalore,Sunrisers Hyderabad,Sunrisers Hyderabad,{'runs': 8}
243813,0,981019,2016,20,2,Sachin Baby,B Kumar,CJ Jordan,0,0,...,0.0,0.0,1,run out,CJ Jordan,['NV Ojha'],Royal Challengers Bangalore,Sunrisers Hyderabad,Sunrisers Hyderabad,{'runs': 8}
243814,0,981019,2016,20,2,Iqbal Abdulla,B Kumar,Sachin Baby,0,1,...,0.0,0.0,0,0,0,0,Royal Challengers Bangalore,Sunrisers Hyderabad,Sunrisers Hyderabad,{'runs': 8}
243815,0,981019,2016,20,2,Sachin Baby,B Kumar,Iqbal Abdulla,1,0,...,0.0,0.0,0,0,0,0,Royal Challengers Bangalore,Sunrisers Hyderabad,Sunrisers Hyderabad,{'runs': 8}


In [55]:
ipl_players_list = []
ipl_players_list.extend(db["batter"].unique().tolist())
ipl_players_list.extend(db["bowler"].unique().tolist())
ipl_players_list.extend(db["non_striker"].unique().tolist())

In [56]:
ipl_players_list = list(set(ipl_players_list))

In [57]:
len(ipl_players_list)

694

In [58]:
df[df["name"] == "TM Head"]

,identifier,cname,name,unique_name
6549,12b610c2,Travis Head,TM Head,TM Head
6550,12b610c2,T Head,TM Head,TM Head


In [59]:
df = df[df["unique_name"].isin(ipl_players_list)]

In [60]:
common_names = df["cname"].tolist()

In [61]:
dfcopy = df.copy()
dfcopy["team"] = [[] for _ in range(len(dfcopy))]

In [63]:
for file in os.listdir(r".\ipl_json"):
    with open(r".\ipl_json"+"\\" + file, "r") as f_input:
        match = json.load(f_input)
        for team in list(match["info"]["players"].keys()):
            for j in match["info"]["players"][team]:
                indices = [ind for ind in df[df["name"] == j].index]
                for index in indices:
                    if team in dfcopy.at[index, "team"]:
                        continue
                    else:
                        teamlist = dfcopy.at[index,"team"]
                        teamlist.append(team)
                        dfcopy.at[index,"team"] = teamlist

In [71]:
dfcopy

,identifier,cname,name,unique_name,team
8,29d72eb2,A Chavan,AA Chavan,AA Chavan,[Rajasthan Royals]
12,2815fe50,A Jhunjhunwala,AA Jhunjhunwala,AA Jhunjhunwala,"[Rajasthan Royals, Pune Warriors, Deccan Charg..."
13,16605a1b,Abrar Kazi,AA Kazi,AA Kazi,[Royal Challengers Bangalore]
35,1c2a64cd,Ashish Reddy,A Ashish Reddy,A Ashish Reddy,"[Deccan Chargers, Sunrisers Hyderabad]"
36,1c2a64cd,AA Reddy,A Ashish Reddy,A Ashish Reddy,"[Deccan Chargers, Sunrisers Hyderabad]"
...,...,...,...,...,...
7071,57ee1fde,Y Chahal,YS Chahal,YS Chahal,"[Royal Challengers Bangalore, Rajasthan Royals..."
7074,1c914163,Y Singh,Yuvraj Singh,Yuvraj Singh,"[Sunrisers Hyderabad, Kings XI Punjab, Mumbai ..."
7077,fe763256,V Rao,Y Venugopal Rao,Y Venugopal Rao,"[Deccan Chargers, Delhi Daredevils, Sunrisers ..."
7080,4e04666c,Y Takawale,YV Takawale,YV Takawale,"[Mumbai Indians, Royal Challengers Bangalore]"


In [72]:
dfcopy.to_csv("names.csv")

In [81]:
from batterstats import display_batter_overall
from bowlerstats import display_bowler_overall

In [82]:
df = pd.read_csv("ipl-ball-by-ball.csv")
names = pd.read_csv("names.csv")

In [83]:
batter_list = []

for name in names["unique_name"].unique():
    temp_list = [name]    
    temp_list.extend(display_batter_overall(df, name))    
    batter_list.append(temp_list)

batter_df = pd.DataFrame(batter_list)

In [84]:
batter_df.to_csv("battingranks.csv")

In [85]:
bowler_list = []

for name in names["unique_name"].unique():
    temp_list = [name]    
    temp_list.extend(display_bowler_overall(df, name))    
    bowler_list.append(temp_list)

bowler_df = pd.DataFrame(bowler_list)

In [86]:
bowler_df.to_csv("bowlingranks.csv")